In [1]:
import sys
sys.path.append('../')

In [2]:
import math
import numpy as np
import scipy.sparse as sp
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

import dgl
import dgl.function as fn

import dgl.data
import networkx as nx

from torchdyn.models import *; from torchdyn.datasets import *
from torchdyn import *

Using backend: pytorch


OSError: libcublas.so.10: cannot open shared object file: No such file or directory

# Neural Graph Differential Equations

## Semi-supervised node classification 

This notebook introduces `Neural GDEs` as a general high-performance model for graph structured data. Notebook `07_graph_differential_equations` is designed from the ground up as an introduction to Neural GDEs and therefore contains ample comments to provide insights on some of our design choices. To be accessible to practicioners/researchers without prior experience on GNNs, we discuss some features of `dgl` as well, one of the PyTorch ecosystems for geometric deep learning.

## Data preparation

In [45]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# seed for repeatability
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

torch.manual_seed(0)
np.random.seed(0)

In [46]:
# dgl offers convenient access to GNN benchmark datasets via `dgl.data`...
# other standard datasets (e.g. Citeseer / Pubmed) are also accessible via the dgl.data
# API. The rest of the notebook is compatible with Cora / Citeseer / Pubmed with minimal
# modification required.
data = dgl.data.CoraDataset()

In [47]:
# Cora is a node-classification datasets with 2708 nodes
X = torch.FloatTensor(data.features).to(device)
Y = torch.LongTensor(data.labels).to(device)

# In transductive semi-supervised node classification tasks on graphs, the model has access to all
# node features but only a masked subset of the labels
train_mask = torch.BoolTensor(data.train_mask)
val_mask = torch.BoolTensor(data.val_mask)
test_mask = torch.BoolTensor(data.test_mask)

num_feats = X.shape[1]
n_classes = data.num_labels

# 140 training samples, 300 validation, 1000 test
n_classes, train_mask.sum().item(), val_mask.sum().item(),test_mask.sum().item()

(7, 140, 300, 1000)

In [48]:
# add self-edge for each node
g = data.graph
g.remove_edges_from(nx.selfloop_edges(g))
g.add_edges_from(zip(g.nodes(), g.nodes()))
g = dgl.DGLGraph(g)
edges = g.edges()
n_edges = g.number_of_edges()

n_edges

13264

In [49]:
# compute diagonal of normalization matrix D according to standard formula
degs = g.in_degrees().float()
norm = torch.pow(degs, -0.5)
norm[torch.isinf(norm)] = 0
# add to dgl.Graph in order for the norm to be accessible at training time
g.ndata['norm'] = norm.unsqueeze(1).to(device)

## Neural GCDE 

As Neural ODEs, GDEs require specification of an ODE function (`ODEFunc`), representing the set of layers that will be called repeatedly by the ODE solver.

Here, we use the convolutional variant of Neural GDEs based on GCNs: `Neural GCDEs`. The only difference with alternative neural GDEs resides in the type of GNN layer utilized in the ODEFunc.

For adaptive step GDEs (dopri5) we increase the hidden dimension to 64 to reduce the stiffness of the ODE and therefore the number of ODEFunc evaluations (`NFE`: Number Function Evaluation)

First, we define the auxiliary GNN model as a standard GCN. Luckily, in this example the graph is static and can thus be assigned during initialization. For varying graphs, additional bookeeping is required.

In [50]:
def accuracy(y_hat:torch.Tensor, y:torch.Tensor):
    preds = torch.max(y_hat, 1)[1]
    return torch.mean((y == preds).float())

In [51]:
class GCNLayer(nn.Module):
    def __init__(self, g:dgl.DGLGraph, in_feats:int, out_feats:int, activation,
                 dropout:int, bias:bool=True):
        super().__init__()
        self.g = g
        self.weight = nn.Parameter(torch.Tensor(in_feats, out_feats))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_feats))
        else:
            self.bias = None
        self.activation = activation
        if dropout:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = 0.
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, h):
        if self.dropout:
            h = self.dropout(h)
        h = torch.mm(h, self.weight)
        # normalization by square root of src degree
        h = h * self.g.ndata['norm']
        self.g.ndata['h'] = h
        self.g.update_all(fn.copy_src(src='h', out='m'),
                          fn.sum(msg='m', out='h'))
        h = self.g.ndata.pop('h')
        # normalization by square root of dst degree
        h = h * self.g.ndata['norm']
        # bias
        if self.bias is not None:
            h = h + self.bias
        if self.activation:
            h = self.activation(h)
        return h

Then, we construct the Neural GDE as follows:

In [52]:
func = nn.Sequential(GCNLayer(g=g, in_feats=64, out_feats=64, activation=nn.Softplus(), dropout=0.9),
                     GCNLayer(g=g, in_feats=64, out_feats=64, activation=None, dropout=0.9)
                     ).to(device)

In [53]:
neuralDE = NeuralDE(func, solver='rk4', s_span=torch.linspace(0, 1, 4)).to(device)

In [54]:
m = nn.Sequential(GCNLayer(g=g, in_feats=num_feats, out_feats=64, activation=F.relu, dropout=0.4),
                  neuralDE,
                  GCNLayer(g=g, in_feats=64, out_feats=n_classes, activation=None, dropout=0.)
                  ).to(device)

## Training loop

In [55]:
class PerformanceContainer(object):
    """ Simple data class for metrics logging."""
    def __init__(self, data:dict):
        self.data = data
        
    @staticmethod
    def deep_update(x, y):
        for key in y.keys():
            x.update({key: list(x[key] + y[key])})
        return x

In [56]:
opt = torch.optim.Adam(m.parameters(), lr=1e-3, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
logger = PerformanceContainer(data={'train_loss':[], 'train_accuracy':[],
                                   'test_loss':[], 'test_accuracy':[],
                                   'forward_time':[], 'backward_time':[],
                                   })


In [57]:
steps = 3000
verbose_step = 150
num_grad_steps = 0

for i in range(steps): # looping over epochs
    m.train()
    outputs = m(X)
    y_pred = outputs
    loss = criterion(y_pred[train_mask], Y[train_mask])
    opt.zero_grad()
    
    start_time = time.time()
    loss.backward()
    
    opt.step()
    num_grad_steps += 1

    with torch.no_grad():
        m.eval()

        # calculating outputs again with zeroed dropout
        y_pred = m(X)

        train_loss = loss.item()
        train_acc = accuracy(y_pred[train_mask], Y[train_mask]).item()
        test_acc = accuracy(y_pred[test_mask], Y[test_mask]).item()
        test_loss = criterion(y_pred[test_mask], Y[test_mask]).item()
        logger.deep_update(logger.data, dict(train_loss=[train_loss], train_accuracy=[train_acc],
                           test_loss=[test_loss], test_accuracy=[test_acc])
                          )

    if num_grad_steps % verbose_step == 0:
        print('[{}], Loss: {:3.3f}, Train Accuracy: {:3.3f}, Test Accuracy: {:3.3f}'.format(num_grad_steps,
                                                                                                    train_loss,
                                                                                                    train_acc,
                                                                                                    test_acc,
                                                                                                    ))

DGLError: [12:24:44] /opt/dgl/src/runtime/c_runtime_api.cc:87: Check failed: allow_missing: Device API gpu is not enabled. Please install the cuda version of dgl.
Stack trace:
  [bt] (0) /home/jyp/michael_dev/testenv/lib/python3.7/site-packages/dgl/libdgl.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x22) [0x7f40f8994dc2]
  [bt] (1) /home/jyp/michael_dev/testenv/lib/python3.7/site-packages/dgl/libdgl.so(dgl::runtime::DeviceAPIManager::GetAPI(std::string, bool)+0x8dd) [0x7f40f8f9c3cd]
  [bt] (2) /home/jyp/michael_dev/testenv/lib/python3.7/site-packages/dgl/libdgl.so(dgl::runtime::DeviceAPI::Get(DLContext, bool)+0x174) [0x7f40f8f98004]
  [bt] (3) /home/jyp/michael_dev/testenv/lib/python3.7/site-packages/dgl/libdgl.so(dgl::runtime::NDArray::Empty(std::vector<long, std::allocator<long> >, DLDataType, DLContext)+0x26e) [0x7f40f8fbd1ce]
  [bt] (4) /home/jyp/michael_dev/testenv/lib/python3.7/site-packages/dgl/libdgl.so(dgl::runtime::NDArray::CopyTo(DLContext const&) const+0xb7) [0x7f40f9026ed